<a href="https://colab.research.google.com/github/SLane35/CollabCompet/blob/master/EntityMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/anhaidgroup/deepmatcher.git

  Cloning https://github.com/anhaidgroup/deepmatcher.git to /tmp/pip-req-build-cr0jnbx1
  Running command git clone -q https://github.com/anhaidgroup/deepmatcher.git /tmp/pip-req-build-cr0jnbx1
     |████████████████████████████████| 71kB 3.6MB/s 
  Created wheel for deepmatcher: filename=deepmatcher-0.1.1-py2.py3-none-any.whl size=55003 sha256=79deb95ea84dd4bccb70c9ef509e1379683d167b3568ae6e794a6a65a56d1db4
  Stored in directory: /tmp/pip-ephem-wheel-cache-qh59odo8/wheels/21/b3/11/dc933837985fcb4683f016ed48f88ae73f90f9b20c122fbedd
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015362 sha256=51d06b58e180a642cb27a274bc56e39e290bac1a2e4a6358e7f4d761097212d5
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built deepmatcher fasttext


In [ ]:

#try:
#    import deepmatcher
#except:
#    !pip install -qqq deepmatcher


In [ ]:
import deepmatcher as dm

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install -qqq recordlinkage

     |████████████████████████████████| 952kB 4.5MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 


In [ ]:
import pandas as pd
import recordlinkage
from recordlinkage.datasets import load_febrl1
df = load_febrl1(True)

In [ ]:
dfNoLabels = load_febrl1()


In [ ]:
matching_dictionary = df[1]

In [ ]:
left_df = dfNoLabels.loc[matching_dictionary.get_level_values(0)]

In [ ]:
right_df = dfNoLabels.loc[matching_dictionary.get_level_values(1)]

In [ ]:
left_df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-223-dup-0,jamilla,wallner,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-dup-0,lachlan,berry,69,giblin street,killarney,bittern,4184,qld,19990219,7364009
rec-373-dup-0,deakin,sondergeld,231,goldfinch circuit,kooltuo,canterbury,2509,vic,19600210,2635962
rec-10-org,kayla,harrington,38,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-dup-0,mia,purdon,23,ramsay place,mirani,gar butt,2206,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
rec-132-org,joshua,ingels,14,arabana street,tatyara,port lincoln,2325,vic,19080503,3603864
rec-498-dup-0,claire,crook,9,oyster farm,basedow street,NaN,3146,vic,NaN,2161417
rec-8-org,james,jesser,53,howe crescent,nuffield village,st clair,4215,vic,19120111,5961736
rec-222-dup-0,emiiyl,coleman,108,chewings street,berkeleyv lge,wellington point,2550,nsw,19421221,7206933


In [ ]:
right_df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
rec-132-dup-0,joshua,ingelbs,14,arabana street,tatyara,port lincoln,2352,vic,19080503,3603864
rec-498-org,claire,crook,9,basedow street,oyster farm,wanniassa,3146,vic,NaN,2161417
rec-8-dup-0,james,jeslser,53,howe crescent,nuffield v illage,st clair,4215,vic,19120111,5961736
rec-222-org,emiily,coleman,108,chewings street,berkeley vlge,wellington point,2550,nsw,19421221,7206933


In [ ]:
left_df = left_df.drop(columns=['street_number', 'address_2', 'postcode', 'soc_sec_id'])
left_df

,given_name,surname,address_1,suburb,state,date_of_birth
rec-223-dup-0,jamilla,wallner,tullaroop street,st james,wa,19081209
rec-122-dup-0,lachlan,berry,giblin street,bittern,qld,19990219
rec-373-dup-0,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210
rec-10-org,kayla,harrington,maltby circuit,coolaroo,nsw,19150612
rec-227-dup-0,mia,purdon,ramsay place,gar butt,vic,19831024
...,...,...,...,...,...,...
rec-132-org,joshua,ingels,arabana street,port lincoln,vic,19080503
rec-498-dup-0,claire,crook,oyster farm,NaN,vic,NaN
rec-8-org,james,jesser,howe crescent,st clair,vic,19120111
rec-222-dup-0,emiiyl,coleman,chewings street,wellington point,nsw,19421221


In [ ]:
right_df = right_df.drop(columns=['street_number', 'address_2', 'postcode', 'soc_sec_id'])
right_df

,given_name,surname,address_1,suburb,state,date_of_birth
rec-223-org,NaN,waller,tullaroop street,st james,wa,19081209
rec-122-org,lachlan,berry,giblin street,bittern,qld,19990219
rec-373-org,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210
rec-10-dup-0,kayla,harrington,maltby circuit,coolaroo,nsw,19150612
rec-227-org,luke,purdon,ramsay place,garbutt,vic,19831024
...,...,...,...,...,...,...
rec-132-dup-0,joshua,ingelbs,arabana street,port lincoln,vic,19080503
rec-498-org,claire,crook,basedow street,wanniassa,vic,NaN
rec-8-dup-0,james,jeslser,howe crescent,st clair,vic,19120111
rec-222-org,emiily,coleman,chewings street,wellington point,nsw,19421221


In [ ]:
left_df_renamed = left_df.rename(columns={'given_name':'left_given_name', 'surname': 'left_surname', 'address_1':'left_address_1', 'suburb':'left_suburb', 'state':'left_state', 'date_of_birth':'left_date_of_birth'})

In [ ]:
right_df_renamed = right_df.rename(columns={'given_name':'right_given_name', 'surname': 'right_surname',  'address_1':'right_address_1', 'suburb':'right_suburb', 'state':'right_state', 'date_of_birth':'right_date_of_birth'})

In [ ]:
right_df_renamed['id'] = range(1, len(right_df_renamed)+1)

In [ ]:
left_df_renamed['id'] = range(1, len(left_df_renamed)+1)

In [ ]:
combined_df = pd.merge(left_df_renamed, right_df_renamed, on="id")

In [ ]:
combined_df['label'] = 1
combined_df

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth,label
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,NaN,waller,tullaroop street,st james,wa,19081209,1
1,lachlan,berry,giblin street,bittern,qld,19990219,2,lachlan,berry,giblin street,bittern,qld,19990219,1
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,1
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,1
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,luke,purdon,ramsay place,garbutt,vic,19831024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,joshua,ingelbs,arabana street,port lincoln,vic,19080503,1
496,claire,crook,oyster farm,NaN,vic,NaN,497,claire,crook,basedow street,wanniassa,vic,NaN,1
497,james,jesser,howe crescent,st clair,vic,19120111,498,james,jeslser,howe crescent,st clair,vic,19120111,1
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,emiily,coleman,chewings street,wellington point,nsw,19421221,1


In [ ]:
right_df_shuffled = right_df.sample(frac=1).reset_index(drop=True)
right_df_shuffled

,given_name,surname,address_1,suburb,state,date_of_birth
0,oliver,rickett,sturt avenue,jamestown,qld,19431020
1,kieren,reid,NaN,kirwsn,nsw,19790121
2,holly,siemers,strachan place,beaudesert,sa,19610319
3,benjamin,campbell,goodenia street,safety beach,qld,19330926
4,jayde,van keulen,macgregor street,cremorne,vic,19150821
...,...,...,...,...,...,...
495,davila-sanhdars,dante,wild street,avondale heights,wa,19120206
496,harriet,belperi,hawker place,roselands,qld,NaN
497,NaN,mcveigh,bougainville street,ourimbh,wa,19360219
498,kirrah,mccarthy,laseron place,winmalee,wa,19400306


In [ ]:
right_df_shuffled = right_df_shuffled.rename(columns={'given_name':'right_given_name', 'surname': 'right_surname',  'address_1':'right_address_1', 'suburb':'right_suburb', 'state':'right_state', 'date_of_birth':'right_date_of_birth'})

In [ ]:
right_df_shuffled['id'] = range(1, len(right_df_shuffled)+1)

In [ ]:
combined_df_mixed = pd.merge(left_df_renamed, right_df_shuffled, on="id")
combined_df_mixed

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,oliver,rickett,sturt avenue,jamestown,qld,19431020
1,lachlan,berry,giblin street,bittern,qld,19990219,2,kieren,reid,NaN,kirwsn,nsw,19790121
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,holly,siemers,strachan place,beaudesert,sa,19610319
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,benjamin,campbell,goodenia street,safety beach,qld,19330926
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,jayde,van keulen,macgregor street,cremorne,vic,19150821
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,davila-sanhdars,dante,wild street,avondale heights,wa,19120206
496,claire,crook,oyster farm,NaN,vic,NaN,497,harriet,belperi,hawker place,roselands,qld,NaN
497,james,jesser,howe crescent,st clair,vic,19120111,498,NaN,mcveigh,bougainville street,ourimbh,wa,19360219
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,kirrah,mccarthy,laseron place,winmalee,wa,19400306


In [ ]:
combined_df_mixed['label'] = 0
combined_df_mixed

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth,label
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,oliver,rickett,sturt avenue,jamestown,qld,19431020,0
1,lachlan,berry,giblin street,bittern,qld,19990219,2,kieren,reid,NaN,kirwsn,nsw,19790121,0
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,holly,siemers,strachan place,beaudesert,sa,19610319,0
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,benjamin,campbell,goodenia street,safety beach,qld,19330926,0
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,jayde,van keulen,macgregor street,cremorne,vic,19150821,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,davila-sanhdars,dante,wild street,avondale heights,wa,19120206,0
496,claire,crook,oyster farm,NaN,vic,NaN,497,harriet,belperi,hawker place,roselands,qld,NaN,0
497,james,jesser,howe crescent,st clair,vic,19120111,498,NaN,mcveigh,bougainville street,ourimbh,wa,19360219,0
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,kirrah,mccarthy,laseron place,winmalee,wa,19400306,0


In [ ]:
final_df = pd.concat([combined_df, combined_df_mixed], axis=0)
final_df

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth,label
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,NaN,waller,tullaroop street,st james,wa,19081209,1
1,lachlan,berry,giblin street,bittern,qld,19990219,2,lachlan,berry,giblin street,bittern,qld,19990219,1
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,1
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,1
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,luke,purdon,ramsay place,garbutt,vic,19831024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,davila-sanhdars,dante,wild street,avondale heights,wa,19120206,0
496,claire,crook,oyster farm,NaN,vic,NaN,497,harriet,belperi,hawker place,roselands,qld,NaN,0
497,james,jesser,howe crescent,st clair,vic,19120111,498,NaN,mcveigh,bougainville street,ourimbh,wa,19360219,0
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,kirrah,mccarthy,laseron place,winmalee,wa,19400306,0


In [ ]:
final_df['id'] = range(1, len(final_df)+1)

In [ ]:
final_df = final_df.sample(frac=1).reset_index(drop=True)

In [ ]:
import numpy as np
train,validate,test = np.split(final_df, [int(.8 * len(final_df)), int(.9 * len(final_df))])

In [ ]:
from google.colab import drive
drive.mount('/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [ ]:
train.to_csv('/drive/My Drive/EntityMatching/train.csv')

In [ ]:
validate.to_csv('/drive/My Drive/EntityMatching/validate.csv')
test.to_csv('/drive/My Drive/EntityMatching/test.csv')

In [ ]:
processed_train, processed_validate, processed_test = dm.data.process(path='/drive/My Drive/EntityMatching', train='train.csv', validation='validate.csv', test='test.csv',ignore_columns=(''), left_prefix='left_',right_prefix='right_',label_attr='label',id_attr='id')


Reading and processing data from "/drive/My Drive/EntityMatching/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/drive/My Drive/EntityMatching/validate.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/drive/My Drive/EntityMatching/test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
train_table = processed_train.get_raw_table()
train_table.head(10)

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth,label
0,joshua,dolan,colborne place,lindfield,nsw,19521116,144,joshua,dolan,colborne place,lindfield,nsw,19521116,1
1,lily,huxley,currie crescent,harristown,qld,19510420,626,,coffey,buller crescent,,vic,19490409,0
2,david,stubbs,wehl place,sunnybank,qld,19490218,239,david,stubbs,wehl place,sunnybank,qld,19490218,1
3,madeleine,paterson,brigalow street,young,nsw,19300302,536,joshua,dolan,colborne place,lindfield,nsw,19521116,0
4,liam,musolino,irvine street,campsie,vic,,347,liam,musloino,irvinewstreet,campsie,vic,,1
5,thomas,guthridge,brunswick circuit,avondale heights,vic,19080625,706,emiily,george,barnett close,modnury,,19861104,0
6,tristan,white,carandini s treet,wilston,nsw,19330405,944,alexandra,boulter,tazewell circuit,potts point,vic,19420722,0
7,jack,rees,mckail crescent,boyanup,qld,19891110,340,jadk,rees,mckail rcescent,boyanup,qld,19891110,1
8,isabelle,lowe,grainger circuit,point turton,vic,19980310,680,sophie,procter,paul coe crescent,port lincoln,sa,19500121,0
9,tori,fleet,tallara parkway,wangaratta,vic,19601101,537,fergua,crofuch,charlotte street,brighton,nsw,19160610,0


# New Section

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    processed_train,
    processed_validate,
    epochs=10,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 13363808
===>  TRAIN Epoch 1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2352: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 1 || Run Time:    8.5 | Load Time:    0.7 || F1:  73.20 | Prec:  58.89 | Rec:  96.70 || Ex/s:  86.96

===>  EVAL Epoch 1


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.6 | Load Time:    0.1 || F1:  95.58 | Prec:  94.74 | Rec:  96.43 || Ex/s: 146.70

* Best F1: tensor(95.5752, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 10 || Run Time:    8.5 | Load Time:    0.7 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  86.52

===>  EVAL Epoch 10


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 10 || Run Time:    0.6 | Load Time:    0.1 || F1:  95.65 | Prec:  93.22 | Rec:  98.21 || Ex/s: 152.19

---------------------

Loading best model...
Training done.


tensor(97.3913, device='cuda:0')

In [ ]:
model.run_eval(processed_test)

===>  EVAL Epoch 4
Finished Epoch 4 || Run Time:    0.3 | Load Time:    0.1 || F1:  97.09 | Prec:  94.34 | Rec: 100.00 || Ex/s: 230.81



tensor(97.0874, device='cuda:0')